In [24]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo
from astropy.cosmology import FlatLambdaCDM

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc


In [25]:
# User input: run folder name 
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

params_path = run_folder / "sncosmo_fits.csv"

df = pd.read_csv(params_path)
print(f"Loaded {len(df)} object(s) from {params_path}")
df.head()

# c : colour paramter, + means redder
# x1 : stretch parameter, + means broader lightcurve
# redshift : calculated from sncosmo fit
# t0 : time of maximum brightness in B-band
# x0 : overall flux scaling parameter
# x1 : stretch parameter, + means broader lightcurve

Loaded 1 object(s) from /Users/david/Code/msc/runs/test1/sncosmo_fits.csv


,ztf_id,redshift,ncall,ndof,chisq,t0,x0,x1,c
0,ZTF19aamhqej,0.016448,112,72,5163.107381,58570.503403,0.014112,-0.73895,-0.011819


In [30]:
# Nuisance parameters
alpha = 0.14
beta = 3.1
M = -19.36
cosmo = FlatLambdaCDM(H0=70.0, Om0=0.3)

# build model once (faster), then set params each loop
model = sncosmo.Model(source="salt3")

ztf_cleansed_path = project_root / "ztf_cleansed.csv"
ztf_df = pd.read_csv(ztf_cleansed_path)
processed_rows = []

for idx, row in df.iterrows():
    ztf_id = row['ztf_id']

    try:
        z = ztf_df.loc[ztf_df['ZTFID'] == ztf_id, 'host_redshift'].iloc[0]
    except IndexError:
        z = row['redshift']

    if pd.isna(z) or z == '' or (isinstance(z, str) and not str(z).strip()):
        z = row['redshift']
    z = float(z)

    ncall = row['ncall']
    ndof = row['ndof']
    chisq = row['chisq']
    t0 = float(row['t0'])
    x0 = float(row['x0'])
    x1 = float(row['x1'])
    c  = float(row['c'])

    # SALT3 safe apparent_mag_B
    model.set(z=z, t0=t0, x0=x0, x1=x1, c=c)
    apparent_mag_B = model.bandmag("bessellb", "ab", model.get("t0"))   # apparent magnitude from rest-frame B
    distance_modulus_u_obs = apparent_mag_B - M # + alpha * x1 - beta * c  # distance modulus after correction
    mu_theoretical  = cosmo.distmod(z).value # distance modulus from cosmology
    resid  = distance_modulus_u_obs - mu_theoretical

    processed_rows.append({
        'ztf_id': ztf_id,
        'host_redshift': z,
        'ncall': ncall,
        'ndof': ndof,
        'chisq': chisq,
        't0': t0,
        'x0': x0,
        'x1': x1,
        'c': c,
        'apparent_mag_B': apparent_mag_B,
        'distance_modulus_u_obs': distance_modulus_u_obs,
        'mu_th': mu_theoretical,
        'resid': resid,
    })

    print(f"{ztf_id}: z={z:.5f}, apparent_mag_B={apparent_mag_B:.3f}, distance_modulus_u_obs={distance_modulus_u_obs:.3f}, mu_theoretical={mu_theoretical:.3f}, resid={resid:.3f}")

output_df = pd.DataFrame(processed_rows)
output_path = run_folder / "distance_process.csv"
output_df.to_csv(output_path, index=False)
print(f"\nWrote processed results to {output_path}")


ZTF19aamhqej: z=0.01645, apparent_mag_B=15.131, distance_modulus_u_obs=34.491, mu_theoretical=34.267, resid=0.224

Wrote processed results to /Users/david/Code/msc/runs/test1/distance_process.csv
